In [1]:
import tensorflow.compat.v1 as tf
import lucid_kietzmannlab.modelzoo.vision_models as models
import matplotlib.pyplot as plt
from tqdm import tqdm
import lucid_kietzmannlab.optvis.objectives as objectives
import lucid_kietzmannlab.optvis.render as render
from ipywidgets import interact, Dropdown, IntSlider
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
model_checkpoint_dir = "/Users/vkapoor/Downloads/training_seed_06"
model_checkpoint = "model.ckpt_epoch89"
model, graph,  layer_shape_dict = models.load_ecoset_model_seeds(model_checkpoint_dir, model_checkpoint)


INFO:tensorflow:Restoring parameters from /Users/vkapoor/Downloads/training_seed_06/model.ckpt_epoch89


2024-06-08 13:40:29.220516: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


Model loaded from: /Users/vkapoor/Downloads/training_seed_06
Layer: tower_0/alexnet_v2/conv1/Conv2D, Shape: (32, 64, 54, 54)
Layer: tower_0/alexnet_v2/conv2/Conv2D, Shape: (32, 192, 26, 26)
Layer: tower_0/alexnet_v2/conv3/Conv2D, Shape: (32, 384, 12, 12)
Layer: tower_0/alexnet_v2/conv4/Conv2D, Shape: (32, 384, 12, 12)
Layer: tower_0/alexnet_v2/conv5/Conv2D, Shape: (32, 256, 12, 12)
Layer: tower_0/alexnet_v2/fc6/Conv2D, Shape: (32, 4096, 1, 1)
Layer: tower_0/alexnet_v2/fc7/Conv2D, Shape: (32, 4096, 1, 1)
Layer: tower_0/alexnet_v2/fc8/Conv2D, Shape: (32, 565, 1, 1)


In [3]:
C = lambda neuron: objectives.channel(*neuron)    
def visualize(layer_name, channel):
    # Check if the layer exists in the shape dictionary
    if layer_name in layer_shape_dict:
        # Check if the selected channel is within bounds
        print(layer_shape_dict[layer_name])
        max_channel = layer_shape_dict[layer_name][-1] - 1
        if 0 <= channel <= max_channel:
            #clear_output(wait=True)
            # Render visualization for the selected layer and channel
            #try:
            _ = render.render_vis(model, C((layer_name, channel)), scope='')
            #except Exception:
            #    print('No gradients for this layer')   

def visualize_all():
    # Check if the layer exists in the shape dictionary
    layer_name = current_dropdown_value({'new': layer_dropdown.value})
    print(layer_name)
    if layer_name in layer_shape_dict:
            # Check if the selected channel is within bounds
            try:
               image_channel = {} 
               for channel in tqdm(range(channel_slider.max)):    
                  images = render.render_vis(model, C((layer_name, channel)), verbose = False)
                  image_channel[channel] = images 
            except Exception:
                print('No gradients for this layer')    
    return image_channel   


# Create dropdown menu for layer selection
layer_dropdown = Dropdown(options=list(layer_shape_dict.keys()), description='Layer:')

# Create slider for channel selection
channel_slider = IntSlider(min=0, max=0, description='Channel:')
        
        
def update_channel_slider(change):
    layer_name = change.new
    if layer_name in layer_shape_dict:
        print(layer_name)
        print(layer_shape_dict[layer_name])
        max_channel = layer_shape_dict[layer_name][-1] - 1
        channel_slider.max = max_channel
        
        
def current_slider_value(*args):
    return channel_slider.value


def current_dropdown_value(change):
    return change['new']


channel_slider.observe(current_slider_value, names='value')
layer_dropdown.observe(current_dropdown_value, names='value')
          

In [4]:
import numpy as np
lname = 'tower_0/alexnet_v2/conv1/Conv2D'
shape = (1,224, 224,3)
array_4d_random = np.random.random(shape)
model.get_activations(lname, array_4d_random)

KeyError: "The name 'import/tower_0/alexnet_v2/conv1/Conv2D:0' refers to a Tensor which does not exist. The operation, 'import/tower_0/alexnet_v2/conv1/Conv2D', does not exist in the graph."

In [5]:
lname = 'tower_0/alexnet_v2/conv1/Conv2D'
visualize(lname, 1)

(32, 64, 54, 54)
tower_0/alexnet_v2/conv1/Conv2D []


KeyError: "The name 'None:0' refers to a Tensor which does not exist. The operation, 'None', does not exist in the graph."

In [ ]:
layer_dropdown.observe(update_channel_slider, names='value')

# Create an interactive visualization
interact(visualize, layer_name=layer_dropdown, channel=channel_slider)     
